Nous allons détailler les fonctions dans ce notebook.

**IMPORT**

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path
import os
import chromedriver_binary

In [2]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"
url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

In [15]:
# driver_path = str(Path("C:/Users/Thomas/chromedriver.exe").resolve())

In [24]:
driver_path2 = Path("C:/Users/Thomas/chromedriver.exe")

In [26]:
str(driver_path2)

'C:/Users/Thomas/chromedriver.exe'

In [4]:
# driver_path= "C:\\Users\Thomas\chromedriver.exe"

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path="C:/Users/Thomas/chromedriver.exe")
# driver.get('http://google.com/')
# print("Chrome Browser Invoked")
# driver.quit()

/tmp/ipykernel_18588/1773416873.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options = options, executable_path="C:/Users/Thomas/chromedriver.exe")
/tmp/ipykernel_18588/1773416873.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path="C:/Users/Thomas/chromedriver.exe")


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [5]:
driver = webdriver.Chrome()
driver.get("https://recipes.lewagon.com/recipes/advanced")


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5606b5c7c233 <unknown>
#1 0x5606b59d2658 <unknown>
#2 0x5606b59f3a39 <unknown>
#3 0x5606b59f120a <unknown>
#4 0x5606b5a2bd4a <unknown>
#5 0x5606b5a25f33 <unknown>
#6 0x5606b59fbc8a <unknown>
#7 0x5606b59fcd55 <unknown>
#8 0x5606b5cacc5d <unknown>
#9 0x5606b5cc617e <unknown>
#10 0x5606b5cae9c5 <unknown>
#11 0x5606b5cc6c95 <unknown>
#12 0x5606b5ca2900 <unknown>
#13 0x5606b5ce1e28 <unknown>
#14 0x5606b5ce1fa8 <unknown>
#15 0x5606b5cfb6ad <unknown>
#16 0x7f93304f3609 <unknown>


In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [/home/thomas/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache
/tmp/ipykernel_18588/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55bd1f1dc7d3 <unknown>
#1 0x55bd1ef38688 <unknown>
#2 0x55bd1ef59a38 <unknown>
#3 0x55bd1ef5768f <unknown>
#4 0x55bd1ef92355 <unknown>
#5 0x55bd1ef8c5a3 <unknown>
#6 0x55bd1ef61ddc <unknown>
#7 0x55bd1ef62de5 <unknown>
#8 0x55bd1f20d49d <unknown>
#9 0x55bd1f22660c <unknown>
#10 0x55bd1f20f205 <unknown>
#11 0x55bd1f226ee5 <unknown>
#12 0x55bd1f203070 <unknown>
#13 0x55bd1f242488 <unknown>
#14 0x55bd1f24260c <unknown>
#15 0x55bd1f25bc6d <unknown>
#16 0x7fa9af3ae609 <unknown>


In [13]:
browser = webdriver.Chrome(executable_path=driver_path)
browser.get(url)

NameError: name 'driver_path' is not defined

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

browser = webdriver.Firefox()

browser.get('http://www.yahoo.com')
assert 'Yahoo' in browser.title

elem = browser.find_element(By.NAME, 'p')  # Find the search box
elem.send_keys('seleniumhq' + Keys.RETURN)

browser.quit()

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


# Trouver l'url des livres sur une page données

In [11]:
response = requests.get(url_kindle_cat_art_and_photo_subcat_umusic_p2)
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
soup

<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Sorry! Something went wrong!</title>
<style>
  html, body {
    padding: 0;
    margin: 0
  }

  img {
    border: 0
  }

  #a {
    background: #232f3e;
    padding: 11px 11px 11px 192px
  }

  #b {
    position: absolute;
    left: 22px;
    top: 12px
  }

  #c {
    position: relative;
    max-width: 800px;
    padding: 0 40px 0 0
  }

  #e, #f {
    height: 35px;
    border: 0;
    font-size: 1em
  

In [ ]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

# Sur une page récupérer les données : de classement de vente , nom du livre catérgorie